In [2]:
import urllib.request
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

***generating image data***

In [ ]:
url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
file_name = "horse-or-human.zip"
training_dir = '../data/horse-or-human/training/'

urllib.request.urlretrieve(url, file_name)
zip_ref = zipfile.ZipFile(file_name,'r')
zip_ref.extractall(training_dir)
zip_ref.close()
